In [1]:
import pandas as pd
import geopandas as gpd

In [2]:
data = gpd.read_file('./data/上海充电桩_中促盟_处理后.geojson')
data

,myid,充电站名称,充电站位置,充电站投入使用时间,充电站所处道路属性,经度,纬度,充电站所属区域分类,充电站类型,交流桩数量,直流桩数量,交流桩总装机功率,直流桩总装机功率,geometry
0,0,上海金山石化十五村充电项目,上海市市辖区金山区石化街道金山城市沙滩石化十五村,2023-09-27,01城市内道路,121.347914,30.712635,01充电场站,05其他专用,6.0,0.0,42.0,0,POINT (121.34791 30.71263)
1,1,上海金山石化十一村小区充电站,上海市市辖区金山区石化街道金山城市沙滩石化十一村,2023-09-27,01城市内道路,121.348137,30.715199,01充电场站,05其他专用,6.0,0.0,42.0,0,POINT (121.34814 30.71520)
2,2,石化隆安市场停车场,石化街道沪杭公路7958号,2021-09-16,01城市内道路,121.344560,30.716703,10大型建筑物配建停车场,01公共,12.0,0.0,42.0,0,POINT (121.34456 30.71670)
3,3,上海欣鑫化工充电站,上海市市辖区金山区海金路366号,2017-01-18,01城市内道路,121.296063,30.717229,01充电场站,01公共,2.0,1.0,14.0,60,POINT (121.29606 30.71723)
4,4,上海顶煦化学充电站,上海市市辖区金山区金山区海金路369号,2017-01-18,01城市内道路,121.296063,30.717229,01充电场站,01公共,5.0,1.0,35.0,15,POINT (121.29606 30.71723)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19612,19612,加枫路充电站,加枫路26号衡山商务停车场,2022-01-13,None,121.605119,31.313403,03公共机构内部停车场,01公共,0.0,4.0,0.0,240,POINT (121.60512 31.31340)
19613,19613,华润新江湾九里充电站,华润新江湾九里,2023-09-23,None,121.520188,31.338381,07交通枢纽公共停车场,01公共,108.0,0.0,756.0,0,POINT (121.52019 31.33838)
19614,19614,巴斯夫-总部大楼（内部）,江心沙路333号浦东基地总部大楼西翼北侧停车场,2021-11-09,None,121.537500,31.345509,03公共机构内部停车场,04环卫物流车专用,37.0,0.0,259.0,0,POINT (121.53750 31.34551)
19615,19615,宝钱公路慢充站,宝钱公路,2023-11-27,None,121.119527,31.362807,05写字楼内部停车场,01公共,2.0,0.0,14.0,0,POINT (121.11953 31.36281)


In [3]:
data.drop(columns=['充电站位置','充电站投入使用时间'],inplace=True)
data['充电站所处道路属性'] = data['充电站所处道路属性'].fillna('unknown')

In [4]:
data_agg = data.groupby(['经度','纬度']).agg({
    '交流桩数量': 'sum',  
    'myid': 'min',
    '充电站所处道路属性':'min',
    '充电站所属区域分类':'min',
    '充电站类型':'min',
    '直流桩数量': 'sum',
    '交流桩总装机功率':'sum',
    '直流桩总装机功率':'sum'
}).reset_index()

In [5]:
data_agg[(data_agg['交流桩数量']>0)&(data_agg['交流桩总装机功率']<=0)]

,经度,纬度,交流桩数量,myid,充电站所处道路属性,充电站所属区域分类,充电站类型,直流桩数量,交流桩总装机功率,直流桩总装机功率
2855,121.271714,31.231862,86.0,13185,01城市内道路,05写字楼内部停车场,01公共,0.0,0.0,0
5529,121.363096,31.356803,3.0,3173,01城市内道路,10大型建筑物配建停车场,05其他专用,1.0,0.0,41
11295,121.496590,31.025233,8.0,5506,01城市内道路,01充电场站,01公共,0.0,0.0,0
11503,121.501607,31.036741,30.0,5872,01城市内道路,01充电场站,01公共,0.0,0.0,0


In [6]:
data_agg['geometry'] = gpd.points_from_xy(data_agg['经度'],data_agg['纬度'])
data_agg = gpd.GeoDataFrame(data_agg,
                            geometry='geometry')
data_agg.to_file('./data/上海充电桩_中促盟_合并站点.geojson')

In [7]:
data_agg

,经度,纬度,交流桩数量,myid,充电站所处道路属性,充电站所属区域分类,充电站类型,直流桩数量,交流桩总装机功率,直流桩总装机功率,geometry
0,120.893281,31.114590,1.0,8668,01城市内道路,02居民区内部停车场,01公共,0.0,7.0,0,POINT (120.89328 31.11459)
1,120.893938,31.110890,0.0,8561,01城市内道路,01充电场站,01公共,1.0,0.0,40,POINT (120.89394 31.11089)
2,120.913980,31.062432,1.0,6740,01城市内道路,02居民区内部停车场,01公共,0.0,7.0,0,POINT (120.91398 31.06243)
3,120.915917,31.030196,8.0,19011,unknown,02居民区内部停车场,01公共,2.0,56.0,60,POINT (120.91592 31.03020)
4,120.916719,31.067247,1.0,6891,01城市内道路,01充电场站,01公共,0.0,7.0,0,POINT (120.91672 31.06725)
...,...,...,...,...,...,...,...,...,...,...,...
18332,121.937159,30.890878,6.0,18922,01城市内道路,01充电场站,05其他专用,0.0,21.0,0,POINT (121.93716 30.89088)
18333,121.939351,30.949531,16.0,4709,01城市内道路,01充电场站,01公共,1.0,112.0,60,POINT (121.93935 30.94953)
18334,121.942315,30.897054,1.0,149,01城市内道路,01充电场站,01公共,2.0,7.0,60,POINT (121.94231 30.89705)
18335,121.943069,30.915995,1.0,4360,01城市内道路,01充电场站,06虚拟桩群,0.0,7.0,0,POINT (121.94307 30.91599)
